In [1]:
import tensorflow as tf
import numpy      as np
import pandas     as pd
import seaborn    as sns

from   tensorflow.keras.models import  Sequential , clone_model
from   tensorflow.keras.layers import  Dense , Activation , InputLayer

from tensorflow.keras.datasets import boston_housing , mnist, fashion_mnist
from tensorflow.keras.models   import Sequential , clone_model , Model, load_model
from tensorflow.keras.layers   import Dense , Activation , InputLayer , Flatten , Input, BatchNormalization, Dropout, Embedding
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras          import optimizers  
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback  
try:
    from keras.utils.np_utils      import to_categorical
except:
    from tensorflow.keras.utils  import to_categorical
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

# 자연어처리
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 이미지 로드
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.datasets          import load_iris , load_breast_cancer , load_digits
from sklearn.model_selection   import train_test_split

import matplotlib.pyplot as plt

import missingno as msno

In [2]:
import tensorflow as tf
import numpy      as np
import pandas     as pd
import seaborn    as sns
import time
import itertools 

from   tensorflow.keras.models import  Sequential , clone_model
from   tensorflow.keras.layers import  Dense , Activation , InputLayer

from tensorflow.keras.datasets import boston_housing , mnist, fashion_mnist
from tensorflow.keras.models   import Sequential , clone_model , Model, load_model
from tensorflow.keras.layers   import Dense , Activation , InputLayer , Flatten , Input, BatchNormalization, Dropout, Embedding
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras          import optimizers  
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback  
try:
    from keras.utils.np_utils      import to_categorical
except:
    from tensorflow.keras.utils  import to_categorical
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

# 자연어처리
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 이미지 로드
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.datasets          import load_iris , load_breast_cancer , load_digits
from sklearn.model_selection   import train_test_split

import matplotlib.pyplot as plt

import missingno as msno

# p-value and vif check
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# RNN
from tensorflow.keras.layers     import SimpleRNN, LSTM

# ml
import sklearn
from   sklearn.model_selection import train_test_split , KFold, StratifiedKFold, cross_val_score, cross_validate, GridSearchCV
from   sklearn.tree            import DecisionTreeClassifier , export_graphviz
from   sklearn.ensemble        import RandomForestClassifier , VotingClassifier

# 데이터 변환 과정과 머신러닝을 연결해주는 파이프라인
from sklearn.pipeline import make_pipeline

 # LogisticRegression : 0, 1 로 나누기 때문에 회귀가 아니라 분류로 본다!
from   sklearn.linear_model    import LogisticRegression , LinearRegression , Ridge, Lasso, ElasticNet, SGDRegressor, LassoCV, ElasticNetCV

from   sklearn.tree            import DecisionTreeRegressor
from   sklearn.ensemble        import RandomForestRegressor, GradientBoostingRegressor
from   xgboost                 import XGBRegressor
from   lightgbm                import LGBMRegressor
from   sklearn.svm             import SVC, SVR
from   sklearn.neighbors       import KNeighborsClassifier

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA, IncrementalPCA

# 분류 쪽 성능평가
from   sklearn.metrics         import accuracy_score , precision_score , recall_score , f1_score , confusion_matrix , precision_recall_curve, roc_curve, roc_auc_score

# 회귀 쪽 성능평가
from   sklearn.metrics         import mean_squared_error, mean_absolute_error, r2_score

from   sklearn.preprocessing   import LabelEncoder , OneHotEncoder , MinMaxScaler , StandardScaler , Binarizer, PolynomialFeatures
from   sklearn.impute          import SimpleImputer

# 분류 알고리즘 만들 때 사용되는 최상위 알고리즘 > 여기서 상속받아서 알고리즘 생성 가능
from   sklearn.base            import BaseEstimator

# 부스팅 알고리즘
import xgboost as xgb
from   xgboost import plot_importance
from   xgboost import XGBClassifier

import lightgbm as light
from lightgbm import LGBMClassifier
from lightgbm import plot_importance

# from imblearn.over_sampling  import SMOTE

In [4]:
# 관측데이터 로드 및 필요한 기온차, 기압차 컬럼추가
male_frm = pd.read_csv('../content/hospital_male_0725.csv', encoding='cp949')
female_frm = pd.read_csv('../content/hospital_male_0725.csv', encoding='cp949')
male_frm["min_max_Ta"] = male_frm["max_ta"]-male_frm["min_ta"]
female_frm["min_max_Ta"] = male_frm["max_ta"]-male_frm["min_ta"]
male_frm["min_max_ps"] = male_frm["max_ps"]-male_frm["min_ps"]
female_frm["min_max_ps"] = male_frm["max_ps"]-male_frm["min_ps"]

In [6]:
# VIF Factor 보기 전 컬럼들 선별작업. 최고저, 차이 기상데이터는 서로 영향을 많이 주므로 나올 수 있는 가짓수 다 넣어봄.
# EDA 결과 차이 기상데이터는 질환발생과 연관이 있다고 판단되어 fix함.

ps_list=[]
ta_list=[]
rhm_list=[]
avg_ps = male_frm[["avg_ps"]]
max_ps = male_frm[["max_ps"]]
min_ps = male_frm[["min_ps"]]
max_ta = male_frm[["max_ta"]]
min_ta = male_frm[["min_ta"]]
avg_ta = male_frm[["avg_ta"]]
avg_rhm = male_frm[["avg_rhm"]]
min_rhm = male_frm[["min_rhm"]]
ps_list.append(avg_ps)
ps_list.append(max_ps)
ps_list.append(min_ps)
ta_list.append(avg_ta)
ta_list.append(max_ta)
ta_list.append(min_ta)
rhm_list.append(avg_rhm)
rhm_list.append(min_rhm)

In [7]:
# VIF Factor 보는데 필요 없는 컬럼 제거(자동화)
vif_male = male_frm.drop(columns=["area", "tma", "Unnamed: 0", "Unnamed: 0.1","yyyy","mm","dd","max_ps", "min_ps","max_ta", "min_ta","avg_ta", "avg_rhm","min_rhm","avg_ps","sex", "D/R","frequency"])
vif_female = female_frm.drop(columns=["area", "tma", "Unnamed: 0", "Unnamed: 0.1","yyyy","mm","dd","max_ps","min_ps", "max_ta", "min_ta", "avg_ta", "avg_rhm", "min_rhm","avg_ps","sex", "D/R","frequency"])

In [25]:
final_result = []
vif_m = pd.DataFrame()
for i in ps_list:
    a = pd.concat([vif_male, i], axis=1)
    for k in ta_list:
        b = pd.concat([a, k], axis = 1)
        for l in rhm_list:
            c = pd.concat([b, l], axis = 1)
            vif_m["VIF Factor"] = [variance_inflation_factor(c.values, n)  for n in range(c.shape[1])]
            vif_m["features"] = c.columns
            vif_sorted_by_index = vif_m.sort_values(by="VIF Factor",ascending=True)
            display( vif_sorted_by_index[vif_sorted_by_index["VIF Factor"]<10])
    c = pd.DataFrame()
    b = pd.DataFrame()
    a = pd.DataFrame()


,VIF Factor,features
0,1.102554,sum_gsr
1,1.322712,sum_rn
11,1.433278,min_max_ps
13,6.633227,avg_ta
4,7.283135,pm10
2,8.200529,so2


,VIF Factor,features
0,1.102554,sum_gsr
1,1.322712,sum_rn
11,1.433278,min_max_ps
13,6.633227,avg_ta
4,7.283135,pm10
2,8.200529,so2


,VIF Factor,features
0,1.102797,sum_gsr
1,1.322519,sum_rn
11,1.433585,min_max_ps
4,7.284368,pm10
2,8.212732,so2


,VIF Factor,features
0,1.102797,sum_gsr
1,1.322519,sum_rn
11,1.433585,min_max_ps
4,7.284368,pm10
2,8.212732,so2


,VIF Factor,features
0,1.102797,sum_gsr
1,1.322519,sum_rn
11,1.433585,min_max_ps
13,4.330570,min_ta
4,7.284368,pm10
2,8.212732,so2


...

,VIF Factor,features
0,1.102482,sum_gsr
1,1.322988,sum_rn
11,2.758558,min_max_ps
13,6.606318,avg_ta
4,7.296290,pm10
2,8.204234,so2


,VIF Factor,features
0,1.102729,sum_gsr
1,1.322782,sum_rn
11,2.758891,min_max_ps
4,7.297504,pm10
2,8.216467,so2


,VIF Factor,features
0,1.102729,sum_gsr
1,1.322782,sum_rn
11,2.758891,min_max_ps
4,7.297504,pm10
2,8.216467,so2


,VIF Factor,features
0,1.102729,sum_gsr
1,1.322782,sum_rn
11,2.758891,min_max_ps
13,4.312946,min_ta
4,7.297504,pm10
2,8.216467,so2


,VIF Factor,features
0,1.102729,sum_gsr
1,1.322782,sum_rn
11,2.758891,min_max_ps
13,4.312946,min_ta
4,7.297504,pm10
2,8.216467,so2
